# Effects of sampling on Phydelity clustering results using A/H3N2 influenza virus sequence data from McCrone et al. (2018)

In a within-host viral diversity study of influneza viruses by [McCrone et al. (2018)](https://elifesciences.org/articles/35962), forty-three high-quality transmission pairs were identified based on stringent criteria:  

1. Only individuals in a household with symptom onset within a 7 day window are considered to be epidemiologically linked.  
2. Transmission event must not have multiple possible donors with the same day of symptom onset.  
3. Donor and recipients were not allowed to have symptom onset on the same day, unless the individuals were both index cases for the household.  
4. Genetic distance between influenza populations from each household pair as measured by L1-norm must be below the 5th percentile of the community distribution of randomly assigned pairs. 

Of the forty-three high-quality transmission pairs, thirty-seven of them were A/H3N2 infections. Among them, thirty-two of them were collected during the 2014/2015 season. This Jupyter notebook extracts the A/H3N2 viral sequences collected by McCrone et al. during that season and analyses how well Phydelity is able to recover the transmission pairs as well as the effect of sampling on Phydelity's results. 


## Import libraries 

In [1]:
from __future__ import division 
import numpy as np
import pandas as pd 
import re
import os
import random
import subprocess 
import itertools 
import datetime
from Bio import SeqIO

import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn.metrics import adjusted_rand_score as ari
from sklearn.metrics import mutual_info_score as mi_score

## Inputs

Before running this Jupyter notebook, we need to download the repository https://github.com/lauringlab/Host_level_IAV_evolution as well as source data 7 csv file for Figure 3 from the online version of the [article](https://elifesciences.org/articles/35962) by McCrone et al. (2018). 

We also need ```mafft``` for sequence alignment, ```RAxML``` to generate ML trees and Phydelity already downloaded and installed. 

In [2]:
# full path to Host_level_IAV_evolution repository 
repo_path = "/Users/alvin/Dropbox/temp_phydelity/Host_level_IAV_evolution" 
# full path to source data 7 csv file
source_data_7_path = "/Users/alvin/Dropbox/github_repo/Phydelity/manuscript/FLU/elife-35962-fig3-data7-v3.csv"

# full path where this Jupyter notebook is downloaded 
root_folder_path = "/Users/alvin/Dropbox/github_repo/Phydelity/manuscript/FLU" 
# path to raxml 
raxml_path = "raxmlHPC-PTHREADS-AVX2"
# path to mafft 
mafft_path = "mafft"

os.chdir(root_folder_path) # make root_folder_path current working directory

## Functions

In [3]:
def parsefasta(fname): # parse fasta file, return as dictionary 
    return {record.name:record.seq for record in SeqIO.parse(fname, "fasta")}

# calculate assessment metrics 
def get_metrics(labels_true, labels_pred):
    
    # sort and count ground truth labels within clusters
    pred_label_to_true_lab_count = {}
    for p, pred_label in enumerate(labels_pred):
        true_label = labels_true[p]
        try:
            pred_label_to_true_lab_count[pred_label][true_label] += 1
        except:
            try:
                pred_label_to_true_lab_count[pred_label][true_label] = 1
            except:
                pred_label_to_true_lab_count[pred_label] = {true_label:1}
    
    # generate confusion matrix 
    confusion_matrix = {}
    for pred_label in pred_label_to_true_lab_count.keys():
        
        for true_label in set(labels_true):
            try:
                count = pred_label_to_true_lab_count[pred_label][true_label]
            except:
                count = 0
            
            try:
                confusion_matrix[true_label].append(count)
            except:
                confusion_matrix[true_label] = [count]
    
    confusion_matrix = pd.DataFrame(confusion_matrix)
    
    # calculate purity using summation of max of each column (class) for each row (cluster) 
    purity = sum(confusion_matrix.max(axis=1))/len(labels_pred)
    
    # modified gini 
    # first, we determine which cluster is the "correct" cluster to class 
    maxc_to_t = {} # nested dictionary of cluster c that subtends the max proportion of class t(s) 
    for t in confusion_matrix:
        # total number of items of class t 
        total_t = sum(confusion_matrix[t])
        
        # get dictionary of cluster index c and the proportion of class t it subtends 
        c_to_tprop = {c:c_count/total_t for c, c_count in enumerate(confusion_matrix[t]) if c_count > 0}
        
        # get the maximum proportion of class t subtended among all clusters 
        max_tprop = max(c_to_tprop.values())
        
        for c, tprop in c_to_tprop.items(): 
            # if cluster c subtends the max. proportion of class t 
            if tprop == max_tprop: 
                try: 
                    maxc_to_t[c][t] = tprop
                except:
                    maxc_to_t[c] = {t:tprop}
    
    # now, we have to make sure that for each class t subtended by cluster c, it is also the largest by 
    # proportion within the cluster 
    t_to_maxc = {}
    for c, t_to_tprop in maxc_to_t.items():
        if len(t_to_tprop) == 1: # cluster only subtends 
            t = t_to_tprop.keys()[0]
            try:
                t_to_maxc[t][c] = tprop
            except: 
                t_to_maxc[t] = {c:tprop}
        else:
            total_counts_in_cluster = sum(confusion_matrix.iloc[c])
            t_to_cprop = {t:list(confusion_matrix[t])[c]/total_counts_in_cluster for t in t_to_tprop.keys()}
            max_cprop = max(t_to_cprop.values())
            
            for t, cprop in t_to_cprop.items():
                if cprop == max_cprop:
                    try:
                        t_to_maxc[t][c] = tprop
                    except: 
                        t_to_maxc[t] = {c:tprop}
    
    # if class t is equally divided across all clusters, then none of the cluster is a "correct" cluster to t 
    for t, c_to_tprop in t_to_maxc.items():
        if len(c_to_tprop) == len(set(labels_pred)) and len(set(c_to_tprop.values())) == 1:
            t_to_maxc[t] = {}
    
    # calculate modified gini
    gini_index = 0 
    for t in list(set(labels_true)):
        c_counts = list(confusion_matrix[t])
        # get proportion of class t among all tips clustered 
        p_t = sum(c_counts)/len(labels_true)
        
        try: 
            # for class t with a "correct" cluster identified 
            maxc_list = t_to_maxc[t].keys()
        except:
            # for class t without a "correct" cluster 
            gini_index += p_t
            continue 
        
        # get proportion of correctly classifed class t 
        p_correct = sum([(c_counts[c]/len(labels_true))/p_t for c in maxc_list])
        
        gini_index += p_t*(1-p_correct)
    
    ari_score = ari(labels_true, labels_pred)
    mi = mi_score(labels_true, labels_pred)
    true_H = entropy([labels_true.count(lab) / len(labels_true) for lab in list(set(labels_true))])
    pred_H = entropy([labels_pred.count(lab) / len(labels_pred) for lab in list(set(labels_pred))])
    nmi = 2*mi/(pred_H + true_H)
    
    return purity, gini_index, ari_score, nmi 
    
# calculate information entropy
def entropy(probability_distribution):
    return -sum([p * np.log(p) for p in probability_distribution])

## Extract transmission pairs and sequence data generated by McCrone et al. (2018)  

In [4]:
# read pairs data 
pairs_df = pd.read_csv(source_data_7_path, index_col=0)
pairs_df["estimated_transmission_date"] = pd.to_datetime(pairs_df["estimated_transmission_date"])
# only consider high-quality transmission pairs of A/H3N2 subtype sampled in 2014/2015 season
pairs_df = pairs_df[(pairs_df["Used"]==True)&(pairs_df["Subtype"]=="A/H3N2")&(pairs_df["estimated_transmission_date"]>=pd.Timestamp(2014, 1, 1))] # H3N2 only 
print "{} high-quality transmission pairs collected in 2014/2015 season.".format(len(pairs_df))
pairs_df.head()

32 high-quality transmission pairs collected in 2014/2015 season.


,Nb,CI,Subtype,donor_sample,recipient_sample,estimated_transmission_date,Donor_age,Recipient_age,minority_isnv,transmitted_minority_isnv,ENROLLID1,ENROLLID2,SPECID1,SPECID2,Used
9,1,1-29,A/H3N2,2014-12-25,2014-12-25,2014-12-24,37.024716,3.616775,2,0,50001,50004,HS1563,HS1564,True
10,1,1-40,A/H3N2,2014-12-25,2014-12-31,2014-12-24,37.024716,6.149339,2,0,50001,50003,HS1563,MH8690,True
11,1,1-39,A/H3N2,2014-12-31,2014-12-25,2014-12-24,6.149339,3.616775,2,0,50003,50004,MH8690,HS1564,True
12,1,1-39,A/H3N2,2014-12-31,2014-12-25,2014-12-24,6.149339,37.024716,2,0,50003,50001,MH8690,HS1563,True
14,1,1-200,A/H3N2,2014-12-25,2014-12-25,2014-12-24,3.616775,37.024716,1,0,50004,50001,HS1564,HS1563,True


Some of the transmission pairs have overlapping donors and recipents. As such, we resolve such pairs into transmission clusters. 

In [5]:
all_transmission_clusters = []

for r, row in pairs_df.iterrows():
    specid1 = row["SPECID1"]
    specid2 = row["SPECID2"]

    paired112 = list(pairs_df[pairs_df["SPECID1"]==specid1]["SPECID2"])    
    paired221 = list(pairs_df[pairs_df["SPECID2"]==specid2]["SPECID1"])
    
    paired122 = list(pairs_df[pairs_df["SPECID1"]==specid2]["SPECID2"])
    paired211 = list(pairs_df[pairs_df["SPECID2"]==specid1]["SPECID1"])
    
    all_specid_in_cluster = list(set(paired122)|set(paired112)|set(paired221)|set(paired211))
    all_specid_in_cluster = sorted(all_specid_in_cluster, key=lambda _: int(re.search("\d+", _).group()))
    all_transmission_clusters.append(tuple(all_specid_in_cluster))

all_transmission_clusters = list(set(all_transmission_clusters))
# get list of all sequences identified involved in high-quality transmission pairs 
all_trans_headers = list(set([k for v in all_transmission_clusters for k in v]))

print "Total number of transmission clusters:", len(all_transmission_clusters)
print {c:len([cl for cl in all_transmission_clusters if len(cl)==c]) for c in set(map(len, all_transmission_clusters))}


Total number of transmission clusters: 22
{2: 16, 3: 6}


## Get meta-information of all A/H3N2 sequence datasets collected in the 2014/2015 season. 

Meta-data file found in Host_level_IAV_evolution repository. 

In [6]:
meta_df = pd.read_csv("{}/data/reference/all_meta.csv".format(repo_path), index_col=0)
meta_df["onset"] = pd.to_datetime(meta_df["onset"])
meta_df["collect"] = pd.to_datetime(meta_df["collect"])
meta_df.head()

,HOUSE_ID,ENROLLID,SPECID,onset,collect,vaccination_status,pcr_result,LAURING_ID,DPI,season,log_copy_num,gc_ul,HIGHSD,sequenced,home_collected
1,1191,300825,M53301,2011-01-01,2011-01-04,1,A/H3N2,1101,3.0,10-11,2.816367,46.799308,N,False,0
2,1191,300828,M53302,2011-01-03,2011-01-04,1,A/H3N2,1102,1.0,10-11,6.486882,219156.158420,N,True,0
3,1191,300827,M53319,2010-12-30,2011-01-05,1,A/H3N2,1103,6.0,10-11,4.608481,2899.701461,N,True,0
4,1300,301236,M53358,2011-01-12,2011-01-13,0,A/H1N1,1104,1.0,10-11,2.104354,9.082920,N,False,0
5,1362,301602,M53375,2011-01-13,2011-01-14,1,A/H3N2,1105,1.0,10-11,2.625722,30.171270,Y,False,0


## Obtain whole genome consensus sequence data for A/H3N2 viruses 

Samples were duplicated in the original analyses. Of which, some of them have differences in frequencies of variant alleles. Here, we analyse which the duplicate sample has the highest frequency of the major alllele and use the consensus sequences derived from that duplicate.

In [7]:
# consensus sequence data from Host_level_IAV_evolution depository 
# read dataframe of duplicated sequences 
dup_df = pd.read_csv("{}/data/processed/secondary/duplicate_sequences.csv".format(repo_path), index_col=0)

# sort minority and majority for duplicated sequences 
specid_to_major_lane = {}
for specid in set(dup_df["SPECID_original"]):
    if specid not in list(pairs_df["SPECID1"]) and specid not in list(pairs_df["SPECID2"]):
        continue 
        
    fil_df = dup_df[dup_df["SPECID_original"]==specid]
    gene_to_major_lane = {}
    for gene in set(fil_df["chr"]):
        for pos in set(fil_df[fil_df["chr"]==gene]["pos"]):
            specid_to_var_to_freq = {}
            for var in set(fil_df[(fil_df["chr"]==gene)&(fil_df["pos"]==pos)]["var"]):
                ffil_df = fil_df[(fil_df["chr"]==gene)&(fil_df["pos"]==pos)&(fil_df["var"]==var)]
                
                try: 
                    specid_to_var_to_freq[ffil_df["SPECID1"].iloc[0]][var] = ffil_df["freq1"].iloc[0]
                except: 
                    specid_to_var_to_freq[ffil_df["SPECID1"].iloc[0]] = {var:ffil_df["freq1"].iloc[0]}
                
                try: 
                    specid_to_var_to_freq[ffil_df["SPECID2"].iloc[0]][var] = ffil_df["freq2"].iloc[0]
                except: 
                    specid_to_var_to_freq[ffil_df["SPECID2"].iloc[0]] = {var:ffil_df["freq2"].iloc[0]}
            
            max_var = list([max(specid_to_var_to_freq[_]) for _ in specid_to_var_to_freq.keys()])[0]
            for s, lane_specid in enumerate(specid_to_var_to_freq.keys()):
                if s == 0: 
                    max_specid_freq = specid_to_var_to_freq[lane_specid][max_var]
                    max_specid = lane_specid
                else: 
                    if specid_to_var_to_freq[lane_specid][max_var] > max_specid_freq:
                        max_specid_freq = specid_to_var_to_freq[lane_specid][max_var]
                        max_specid = lane_specid
            
            gmajor_lane = max_specid

            try: 
                gene_to_major_lane[gene][gmajor_lane] += 1
            except: 
                try: 
                    gene_to_major_lane[gene][gmajor_lane] = 1
                except:
                    gene_to_major_lane[gene] = {gmajor_lane:1}

    major_lane_list = [max(v) for k, v in gene_to_major_lane.items()]
    try: 
        specid_to_major_lane[specid] = re.search("HK_\d+", max(set(major_lane_list), key=lambda _: major_lane_list.count(_))).group()
    except: 
        continue 
        
print (specid_to_major_lane)

{'HS1402': 'HK_6', 'MH8159': 'HK_8', 'HS1518': 'HK_7', 'HS1519': 'HK_7', 'HS1417': 'HK_6', 'HS1535': 'HK_7', 'HS1375': 'HK_6', 'HS1516': 'HK_7', 'MH7440': 'HK_7', 'HS1345': 'HK_6', 'MH7953': 'HK_8', 'MH8090': 'HK_8', 'MH7564': 'HK_7', 'HS1543': 'HK_7', 'HS1465': 'HK_6', 'MH8924': 'HK_2', 'MH7390': 'HK_7', 'HS1536': 'HK_7', 'HS1341': 'HK_6', 'MH8160': 'HK_8', 'MH8386': 'HK_8', 'HS1409': 'HK_6', 'MH7843': 'HK_8', 'MH8126': 'HK_8', 'MH7885': 'HK_8', 'MH8089': 'HK_8', 'MH7687': 'HK_7', 'MH8309': 'HK_8', 'MH8690': 'HK_2', 'HS1416': 'HK_6'}


We then retrieve the consensus whole genome sequence data. 

In [8]:
specid_to_index = {}
specid_to_onset_date = {}
specid_to_collect_date = {}

total_count = 0
for folder in os.listdir("{}/data/processed/".format(repo_path)):
    try: 
        lane = re.search("HK_\d+", folder).group()
    except: 
        continue

    for fname in os.listdir("{}/data/processed/{}/parsed_fa".format(repo_path, folder)):
        try: 
            specid, index = fname.split(".")[0].split("_")
        except: 
            continue 
        
        subtype = meta_df[meta_df["SPECID"]==specid]["pcr_result"].iloc[0]
        onset_date = meta_df[meta_df["SPECID"]==specid]["onset"].iloc[0]
        collect_date = meta_df[meta_df["SPECID"]==specid]["collect"].iloc[0]        
            
        # ensure that we are retrieving H3N2 sequences with onset date >= 2014 
        if subtype != "H3N2" or onset_date < pd.Timestamp(2014, 1, 1):
            print (specid, subtype, onset_date)
            continue
        
        specid_to_onset_date[specid] = onset_date
        specid_to_collect_date[specid] = collect_date
        
        try: 
            specid_to_index[specid].append((index,folder))
        except: 
            specid_to_index[specid] = [(index,folder)]

print "{} A/H3N2 viruses sequenced in total from {} households.".format(len(specid_to_index), 
                                                                        len(set(meta_df[meta_df["SPECID"].isin(specid_to_onset_date.keys())]["HOUSE_ID"])))
print "Sample with earliest onset:", min(specid_to_onset_date), "({})".format(specid_to_onset_date[min(specid_to_onset_date)].date())
print "Sample with latest onset:", max(specid_to_onset_date), "({})".format(specid_to_onset_date[max(specid_to_onset_date)].date())
            
for specid, lanes in specid_to_index.items():
    if specid in specid_to_major_lane:
        major_lane_folder = specid_to_major_lane[specid]
        specid_to_index[specid] = [_ for _ in lanes if _[-1] == major_lane_folder]

# get sequences 
if not os.path.isfile("./concatenated_seqeunces_2014_H3N2.fasta"):
    segments = ['PB2', 'PB1', 'PA', 'HA', 'NP', 'NR', 'M', 'NS']
    with open("concatenated_seqeunces_2014_H3N2.fasta", "w") as output: 
        for specid, lanes in specid_to_index.items():
            index, folder = lanes[0]
            fname = ("{}/data/processed/{}/parsed_fa/{}_{}.removed.parsed.fasta".format(repo_path, folder, specid, index))
            fdat = parsefasta(fname)
            # concatenate gene segments 
            concatseq = "".join([str(fdat[g]) for g in segments])
            output.write(">{}\n{}\n".format(specid, concatseq))            

206 A/H3N2 viruses sequenced in total from 81 households.
Sample with earliest onset: HS1250 (2014-11-10)
Sample with latest onset: MH9547 (2015-01-27)


## Build phylogenetic tree and run Phydelity 

We construct the phylogenetic tree for all of the sampled A/H3N2 viruses using RAxML (GTRGAMMA) and apply Phydelity for transmission clustering. 

In [10]:
tree_folder_path = "{}/raxml".format(root_folder_path) # must be full path
phydelity_folder_path = "{}/phydelity".format(tree_folder_path) # must be full path

# make dir for tree_folder_path 
if not os.path.isdir(tree_folder_path):
    os.mkdir(tree_folder_path)
    
# mafft to align sequences 
if not os.path.isfile("./mafft_concatenated_seqeunces_2014_H3N2.fasta"):
    subprocess.call("{} concatenated_seqeunces_2014_H3N2.fasta > mafft_concatenated_seqeunces_2014_H3N2.fasta".format(mafft_path))

# raxml 
if not os.path.isfile("{}/RAxML_bestTree.mccrone_h3n2_2014".format(tree_folder_path)):
    cmd = [raxml_path, "-T", "4", "-m", "GTRGAMMA", "-p", "666", "-#", "10",
           "-s", "mafft_concatenated_seqeunces_2014_H3N2.fasta", 
           "-n", "mccrone_h3n2_2014", "-w", tree_folder_path]
    subprocess.call(cmd)

# apply phydelity 
try:
    phydelity_clstr_file = [fname for fname in os.listdir(phydelity_folder_path) if re.search("^cluster_phydelity_", fname)][0]
except: 
    os.mkdir(phydelity_folder_path)
    os.chdir(phydelity_folder_path)
    subprocess.call("cp {}/RAxML_bestTree.mccrone_h3n2_2014 ./".format(tree_folder_path), shell=True)
    cmd = ["phydelity.py", "--tree", "RAxML_bestTree.mccrone_h3n2_2014", 
           "--outgroup", min(specid_to_onset_date), "--collapse_zero_branch_length"]
    subprocess.call(cmd)
    os.chdir(root_folder_path)
    phydelity_clstr_file = [fname for fname in os.listdir(phydelity_folder_path) if re.search("^cluster_phydelity_", fname)][0]
print (phydelity_clstr_file)

cluster_phydelity_k2_sol0_RAxML_bestTree.txt


## Analyse and calculate clustering metrics 

We then assess the clustering results for the transmission pairs identified by McCrone et al using the same metrics that was used for the simulation runs. Four metrics are computed: 

1. Purity = Average proportion of members in an inferred cluster belonging to the same true cluster.  
2. Modified gini index = Probability of a randomly selected member is incorrectly clustered.  
3. Adjusted rand index (ARI) = Corrected-for-chance accuracy of clustering results based on how well combinatorial pairs of members match between their inferred and true clusters; ranges between 0 and 1; the higher the ARI, the more accurate the clustering results.  
4. Nomalised mutual information (NMI) = measures the trade-off between clustering quality and number of clusters; ranges between 0 and 1; the lower the NMI, the more is clustering completely random. 

Due the stringency of how transmission pairs were identified by McCrone et al., we have also analysed the clustering results using household ID of the sequences as ground truth clusters. 

In [11]:
phydelity_result_all = pd.read_csv("{}/{}".format(phydelity_folder_path, phydelity_clstr_file), delimiter="\t")
phydelity_result_all["TRUE"] = None

unclustered_list = list(set(specid_to_collect_date.keys())-set(phydelity_result_all["TAXA"]))
unclustered_df = {"TAXA":unclustered_list, "CLUSTER":[None]*len(unclustered_list), "TRUE":[None]*len(unclustered_list)}
# generate meta-data csv file with unclustered sequences to generate tree figure 
phydelity_result_meta = pd.concat([phydelity_result_all, pd.DataFrame(unclustered_df)], ignore_index=True)

trans_header_to_tc = {}
for _tc, true_cluster in enumerate(all_transmission_clusters):
    for trans_header in true_cluster: 
        trans_header_to_tc[trans_header] = _tc
        
        try: 
            _index = phydelity_result_meta[phydelity_result_meta["TAXA"]==trans_header].index[0]
            phydelity_result_meta.at[_index, "TRUE"] = _tc
        except: 
            pass 
        
        try: 
            _index = phydelity_result_all[phydelity_result_all["TAXA"]==trans_header].index[0]
            phydelity_result_all.at[_index, "TRUE"] = _tc
        except: 
            continue

phydelity_result_all["HOUSE_ID"] = None
for r, row in phydelity_result_all.iterrows():
    phydelity_result_all.at[r, "HOUSE_ID"] = int(meta_df[meta_df["SPECID"]==row["TAXA"]]["HOUSE_ID"].iloc[0])

# meta dataframe 
phydelity_result_meta["HOUSE_ID"] = None
for r, row in phydelity_result_meta.iterrows():
    phydelity_result_meta.at[r, "HOUSE_ID"] = int(meta_df[meta_df["SPECID"]==row["TAXA"]]["HOUSE_ID"].iloc[0])
phydelity_result_meta = phydelity_result_meta[["TAXA", "CLUSTER", "TRUE", "HOUSE_ID"]]
#phydelity_result_meta = phydelity_result_meta.set_index("TAXA")
phydelity_result_meta = phydelity_result_meta.rename(columns={"TAXA":"leaf", "CLUSTER": "Phydelity", "TRUE": "tp", "HOUSE_ID":"Household"})
phydelity_result_meta.to_csv("phydelity_result_meta.csv", index_label=False)
    
tabulated_results_all = {"Analysis-ID":[], "Basis":[], "Purity":[], "Gini":[], "ARI":[], "NMI":[]}

purity, gini, ari_score, nmi = get_metrics(labels_pred=list(phydelity_result_all["CLUSTER"]),
                                           labels_true=list(phydelity_result_all["HOUSE_ID"]))
tabulated_results_all["Analysis-ID"].append("ALL")
tabulated_results_all["Basis"].append("HOUSEHOLD")
tabulated_results_all["Purity"].append(purity)
tabulated_results_all["Gini"].append(gini)
tabulated_results_all["ARI"].append(ari_score)
tabulated_results_all["NMI"].append(nmi)

purity, gini, ari_score, nmi = get_metrics(labels_pred=list(phydelity_result_all[~pd.isnull(phydelity_result_all["TRUE"])]["CLUSTER"]),
                                           labels_true=list(phydelity_result_all[~pd.isnull(phydelity_result_all["TRUE"])]["TRUE"]))
tabulated_results_all["Analysis-ID"].append("ALL")
tabulated_results_all["Basis"].append("TRANSMISSION_PAIRS")
tabulated_results_all["Purity"].append(purity)
tabulated_results_all["Gini"].append(gini)
tabulated_results_all["ARI"].append(ari_score)
tabulated_results_all["NMI"].append(nmi)

tabulated_results_all = pd.DataFrame(tabulated_results_all)[["Analysis-ID", "Basis", "Purity", "Gini", "ARI", "NMI"]]
tabulated_results_all


,Analysis-ID,Basis,Purity,Gini,ARI,NMI
0,ALL,HOUSEHOLD,0.893750,0.081250,0.790965,0.963980
1,ALL,TRANSMISSION_PAIRS,0.977778,0.022222,0.961926,0.993025


## Downsampling the data

McCrone et al. collected 206 sequences in total. To analyse the effects to clustering results due to lower sampling rates without bias from small sample effects, we randomly downsample the available dataset to either 52 (25%) or 93 (45%) sequences. 

For effective analysis, we need to ensure that there are some high-quality transmission pairs within the downsampled pool. As such, we varied the proportion (25%, 45% or 70%) of each downsampled pool of sequences consisting of isolates making up the high-quality transmission pairs. 10 distinct random downsamples were generated for each downsample pool/high-quality transmission sequences set and the averaged clustering results were compared against the case where all sequences collected were analysed. 

In [12]:
total_number_of_downsampled_sequences = [52, 93] 
proportion_of_trans_headers = [25, 45, 70] # percent
total_replicates = 10
min_true_clusters = 3

fdat = parsefasta("./mafft_concatenated_seqeunces_2014_H3N2.fasta")

tabulated_results_subset = {"Analysis-ID":[], "replicate":[], "Basis":[], "Purity":[], "Gini":[], "ARI":[], "NMI":[]}

for rep in range(total_replicates):
    seed_used = []
    for ds, th in itertools.product(total_number_of_downsampled_sequences, proportion_of_trans_headers):
        if ds == 93 and th == 70: # only perform 70% analysis for n_seq =52
            continue 
            
        # downsample sequences 
        if os.path.isfile("mafft_subset{}_{}_r{}.fasta".format(ds, th, rep)):
            tfdat = parsefasta("mafft_subset{}_{}_r{}.fasta".format(ds, th, rep))
            downsampled_trans_headers = pd.DataFrame({header:trans_header_to_tc[header] for header in list(set(tfdat.keys())&set(all_trans_headers))}.items(), columns=["TAXA", "TRUE"]).sort_values(by="TRUE") 
            number_of_clusters_in_ds = len([k for k, v in {n:list(downsampled_trans_headers["TRUE"]).count(n) for n in set(downsampled_trans_headers["TRUE"])}.items() if v >= 2])
            n_seq = len(tfdat)
        else: 
            curr_seed = random.choice(range(10000)) 
            while curr_seed in seed_used:
                curr_seed = random.choice(range(10000)) 
            seed_used.append(curr_seed)
            random.seed(curr_seed)
            
            downsampled_trans_headers = pd.DataFrame({header:trans_header_to_tc[header] for header in random.sample(all_trans_headers, int(ds*(th/100)))}.items(), columns=["TAXA", "TRUE"]).sort_values(by="TRUE")
            # make sure we have at least 3 true clusters in downsampled_trans_headers 
            number_of_clusters_in_ds = len([k for k, v in {n:list(downsampled_trans_headers["TRUE"]).count(n) for n in set(downsampled_trans_headers["TRUE"])}.items() if v >= 2])

            while number_of_clusters_in_ds < min_true_clusters:
                curr_seed = random.choice(range(10000)) 
                while curr_seed in seed_used:
                    curr_seed = random.choice(range(10000)) 
                seed_used.append(curr_seed)
                random.seed(curr_seed)
                downsampled_trans_headers = pd.DataFrame({header:trans_header_to_tc[header] for header in random.sample(all_trans_headers, int(ds*(th/100)))}.items(), columns=["TAXA", "TRUE"]).sort_values(by="TRUE")
                number_of_clusters_in_ds = len([k for k, v in {n:list(downsampled_trans_headers["TRUE"]).count(n) for n in set(downsampled_trans_headers["TRUE"])}.items() if v >= 2])
            #print (ds, th, rep, curr_seed, number_of_clusters_in_ds)

            downsampled_headers = random.sample(list(set(specid_to_collect_date.keys())-set(all_trans_headers)), ds-len(downsampled_trans_headers))
            downsampled_headers += list(downsampled_trans_headers["TAXA"])
            n_seq = len(downsampled_headers)

            with open("mafft_subset{}_{}_r{}.fasta".format(ds, th, rep), "w") as output: 
                for header in downsampled_headers: 
                    output.write(">{}\n{}\n".format(header, fdat[header]))

        tree_folder_path = "{}/subset{}_{}".format(root_folder_path, ds, th) # must be full path
        rep_folder_path = "{}/r{}".format(tree_folder_path, rep)
        phydelity_folder_path = "{}/phydelity".format(rep_folder_path) # must be full path

        # make dir for tree_folder_path 
        if not os.path.isdir(tree_folder_path):
            os.mkdir(tree_folder_path)
        
        # make dir for rep_folder_path
        if not os.path.isdir(rep_folder_path):
            os.mkdir(rep_folder_path)

        # raxml 
        if not os.path.isfile("{}/RAxML_bestTree.subset{}_{}".format(rep_folder_path, ds, th)):
            # root tree to isolate with earliest onset
            outgroup_header = min({header:specid_to_onset_date[header] for header in downsampled_headers})
            # run raxml
            cmd = [raxml_path, "-T", "4", "-m", "GTRGAMMA", "-p", "666", "-#", "10",
                   "-s", "mafft_subset{}_{}_r{}.fasta".format(ds, th, rep), 
                   "-n", "subset{}_{}".format(ds, th), "-w", rep_folder_path]
            subprocess.call(cmd)

        # apply phydelity 
        try:
            phydelity_clstr_file = [fname for fname in os.listdir(phydelity_folder_path) if re.search("^cluster_phydelity_", fname)][0]
        except: 
            if not os.path.isdir(phydelity_folder_path):
                os.mkdir(phydelity_folder_path)
            os.chdir(phydelity_folder_path)
            subprocess.call("cp {}/RAxML_bestTree.subset{}_{} ./".format(rep_folder_path, ds, th), shell=True)
            cmd = ["phydelity.py", "--tree", "RAxML_bestTree.subset{}_{}".format(ds, th), 
                   "--outgroup", outgroup_header, "--collapse_zero_branch_length", "--pdf_tree"]
            subprocess.call(cmd)
            os.chdir(root_folder_path)
            phydelity_clstr_file = [fname for fname in os.listdir(phydelity_folder_path) if re.search("^cluster_phydelity_", fname)][0]

        phydelity_result_subset = pd.read_csv("{}/{}".format(phydelity_folder_path, phydelity_clstr_file), delimiter="\t")
        phydelity_result_subset["TRUE"] = None

        for _tc, true_cluster in enumerate(all_transmission_clusters):
            for _ in true_cluster: 
                try: 
                    _index = phydelity_result_subset[phydelity_result_subset["TAXA"]==_].index[0]
                    phydelity_result_subset.at[_index, "TRUE"] = _tc
                except: 
                    continue

        phydelity_result_subset["HOUSE_ID"] = None
        for r, row in phydelity_result_subset.iterrows():
            phydelity_result_subset.at[r, "HOUSE_ID"] = int(meta_df[meta_df["SPECID"]==row["TAXA"]]["HOUSE_ID"].iloc[0])

        purity, gini, ari_score, nmi = get_metrics(labels_pred=list(phydelity_result_subset["CLUSTER"]),
                                                  labels_true=list(phydelity_result_subset["HOUSE_ID"]))
        tabulated_results_subset["Analysis-ID"].append("{}-{}".format(ds, th))
        tabulated_results_subset["replicate"].append(rep)
        tabulated_results_subset["Basis"].append("HOUSEHOLD")    
        tabulated_results_subset["Purity"].append(purity)
        tabulated_results_subset["Gini"].append(gini)
        tabulated_results_subset["ARI"].append(ari_score)
        tabulated_results_subset["NMI"].append(nmi)

        purity, gini, ari_score, nmi = get_metrics(labels_pred=list(phydelity_result_subset[~pd.isnull(phydelity_result_subset["TRUE"])]["CLUSTER"]), 
                                                  labels_true=list(phydelity_result_subset[~pd.isnull(phydelity_result_subset["TRUE"])]["TRUE"]))
        tabulated_results_subset["Analysis-ID"].append("{}-{}".format(ds, th))
        tabulated_results_subset["replicate"].append(rep)
        tabulated_results_subset["Basis"].append("TRANSMISSION_PAIRS")
        tabulated_results_subset["Purity"].append(purity)
        tabulated_results_subset["Gini"].append(gini)
        tabulated_results_subset["ARI"].append(ari_score)
        tabulated_results_subset["NMI"].append(nmi)

tabulated_results_subset = pd.DataFrame(tabulated_results_subset)[["Analysis-ID", "replicate", "Basis", "Purity", "Gini", "ARI", "NMI"]]
tabulated_results_subset.head()

,Analysis-ID,replicate,Basis,Purity,Gini,ARI,NMI
0,52-25,0,HOUSEHOLD,0.378378,0.567568,0.104510,0.681016
1,52-25,0,TRANSMISSION_PAIRS,0.727273,0.181818,0.506284,0.867313
2,52-45,0,HOUSEHOLD,0.729730,0.162162,0.520666,0.912625
3,52-45,0,TRANSMISSION_PAIRS,1.000000,0.000000,1.000000,1.000000
4,52-70,0,HOUSEHOLD,0.897436,0.051282,0.846377,0.970116


In [13]:
averaged_tabulated_results_subset = {"Analysis-ID":[], "Basis":[], "Purity":[], "Gini":[], "ARI":[], "NMI":[]}

for analysis_id in set(tabulated_results_subset["Analysis-ID"]):
    fil_df = tabulated_results_subset[tabulated_results_subset["Analysis-ID"]==analysis_id]
    for basis in set(fil_df["Basis"]):
        basis_fil_df = fil_df[fil_df["Basis"]==basis]
        
        averaged_tabulated_results_subset["Analysis-ID"].append(analysis_id)
        averaged_tabulated_results_subset["Basis"].append(basis)
        averaged_tabulated_results_subset["Purity"].append(np.mean(basis_fil_df["Purity"]))
        averaged_tabulated_results_subset["Gini"].append(np.mean(basis_fil_df["Gini"]))
        averaged_tabulated_results_subset["ARI"].append(np.mean(basis_fil_df["ARI"]))
        averaged_tabulated_results_subset["NMI"].append(np.mean(basis_fil_df["NMI"]))

averaged_tabulated_results_subset = pd.DataFrame(averaged_tabulated_results_subset)
averaged_tabulated_results_subset

,ARI,Analysis-ID,Basis,Gini,NMI,Purity
0,0.800741,93-45,HOUSEHOLD,0.108466,0.953457,0.868013
1,0.898362,93-45,TRANSMISSION_PAIRS,0.029919,0.980917,0.942437
2,0.345184,52-25,HOUSEHOLD,0.285097,0.817225,0.562839
3,0.717998,52-25,TRANSMISSION_PAIRS,0.061515,0.932114,0.870808
4,0.744397,52-70,HOUSEHOLD,0.113450,0.928044,0.818475
5,0.761619,52-70,TRANSMISSION_PAIRS,0.069640,0.943068,0.850879
6,0.563890,52-45,HOUSEHOLD,0.158058,0.898070,0.726313
7,0.737286,52-45,TRANSMISSION_PAIRS,0.040606,0.950013,0.870330
8,0.635009,93-25,HOUSEHOLD,0.163730,0.918509,0.754083
9,0.881796,93-25,TRANSMISSION_PAIRS,0.026836,0.979408,0.943546


In [14]:
combined_results = pd.concat([tabulated_results_all, averaged_tabulated_results_subset], ignore_index=True).sort_values(by="Analysis-ID")
combined_results = combined_results[["Analysis-ID", "Basis", "Purity", "Gini", "ARI", "NMI"]]
combined_results.to_csv("combined_results.csv", index=False)
combined_results

/Users/alvin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Analysis-ID,Basis,Purity,Gini,ARI,NMI
4,52-25,HOUSEHOLD,0.562839,0.285097,0.345184,0.817225
5,52-25,TRANSMISSION_PAIRS,0.870808,0.061515,0.717998,0.932114
8,52-45,HOUSEHOLD,0.726313,0.158058,0.563890,0.898070
9,52-45,TRANSMISSION_PAIRS,0.870330,0.040606,0.737286,0.950013
6,52-70,HOUSEHOLD,0.818475,0.113450,0.744397,0.928044
7,52-70,TRANSMISSION_PAIRS,0.850879,0.069640,0.761619,0.943068
10,93-25,HOUSEHOLD,0.754083,0.163730,0.635009,0.918509
11,93-25,TRANSMISSION_PAIRS,0.943546,0.026836,0.881796,0.979408
2,93-45,HOUSEHOLD,0.868013,0.108466,0.800741,0.953457
3,93-45,TRANSMISSION_PAIRS,0.942437,0.029919,0.898362,0.980917


In [15]:
combined_results[combined_results["Basis"]=="TRANSMISSION_PAIRS"]

,Analysis-ID,Basis,Purity,Gini,ARI,NMI
5,52-25,TRANSMISSION_PAIRS,0.870808,0.061515,0.717998,0.932114
9,52-45,TRANSMISSION_PAIRS,0.870330,0.040606,0.737286,0.950013
7,52-70,TRANSMISSION_PAIRS,0.850879,0.069640,0.761619,0.943068
11,93-25,TRANSMISSION_PAIRS,0.943546,0.026836,0.881796,0.979408
3,93-45,TRANSMISSION_PAIRS,0.942437,0.029919,0.898362,0.980917
1,ALL,TRANSMISSION_PAIRS,0.977778,0.022222,0.961926,0.993025


In [16]:
combined_results[combined_results["Basis"]=="HOUSEHOLD"]

,Analysis-ID,Basis,Purity,Gini,ARI,NMI
4,52-25,HOUSEHOLD,0.562839,0.285097,0.345184,0.817225
8,52-45,HOUSEHOLD,0.726313,0.158058,0.563890,0.898070
6,52-70,HOUSEHOLD,0.818475,0.113450,0.744397,0.928044
10,93-25,HOUSEHOLD,0.754083,0.163730,0.635009,0.918509
2,93-45,HOUSEHOLD,0.868013,0.108466,0.800741,0.953457
0,ALL,HOUSEHOLD,0.893750,0.081250,0.790965,0.963980
